## 03-Homework
---

In [13]:
from google.cloud import bigquery
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../keys/service-account.json" 

client_query = bigquery.Client()

---

### Put parquet files in the bucket

In [11]:
from google.cloud import storage
import os

# Path to your service account key
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../keys/service-account.json"

bucket_name = "kestra-bucket-484712a"
folder_path = "../parquet_files"

client = storage.Client()
bucket = client.bucket(bucket_name)

# Upload all parquet files in folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".parquet"):
        blob = bucket.blob(file_name)
        blob.upload_from_filename(os.path.join(folder_path, file_name))
        print(f"Uploaded {file_name} to {bucket_name}")


Uploaded yellow_tripdata_2024-03.parquet to kestra-bucket-484712a
Uploaded yellow_tripdata_2024-05.parquet to kestra-bucket-484712a
Uploaded yellow_tripdata_2024-02.parquet to kestra-bucket-484712a
Uploaded yellow_tripdata_2024-04.parquet to kestra-bucket-484712a
Uploaded yellow_tripdata_2024-06.parquet to kestra-bucket-484712a
Uploaded yellow_tripdata_2024-01.parquet to kestra-bucket-484712a


### Create External Table

In [ ]:
query = """
CREATE OR REPLACE EXTERNAL TABLE `kestra-sandbox-485712.zoomcamp.external_yellow_tripdata_parquet`
OPTIONS (
  format = 'PARQUET',
  uris = ['gs://kestra-bucket-484712a/*.parquet']
);
"""

client_query.query(query).to_dataframe()

### Create Regular materialized Table from above External table

In [ ]:
query = """
CREATE OR REPLACE TABLE `kestra-sandbox-485712.zoomcamp.yellow_tripdata_01_to_06_2024`
AS
SELECT *
FROM `kestra-sandbox-485712.zoomcamp.external_yellow_tripdata_parquet`;
"""

client_query.query(query).to_dataframe()

---

#### Question 1:
- What is count of records for the 2024 Yellow Taxi Data?

In [14]:
query = """
SELECT COUNT(*) AS total_rows
FROM `kestra-sandbox-485712.zoomcamp.yellow_tripdata_01_to_06_2024`;
"""

client_query.query(query).to_dataframe()

,total_rows
0,20332093


---

#### Question 2:
- Write a query to count the distinct number of PULocationIDs for the entire dataset on both the tables.
- What is the estimated amount of data that will be read when this query is executed on the External Table and the Table?


- Answer: 0 MB for the External Table and 155.12 MB for the Materialized Table

In [ ]:
query = """
SELECT COUNT(DISTINCT PULocationID)
FROM `kestra-sandbox-485712.zoomcamp.external_yellow_tripdata_parquet`;

SELECT COUNT(DISTINCT PULocationID)
FROM `kestra-sandbox-485712.zoomcamp.yellow_tripdata_01_to_06_2024`;
"""

client_query.query(query).to_dataframe()

---

#### Question 3:
- Write a query to retrieve the PULocationID from the table (not the external table) in BigQuery. 
- Now write a query to retrieve the PULocationID and DOLocationID on the same table. 
- Why are the estimated number of Bytes different?

- Answer: BigQuery duplicates data across multiple storage partitions, so selecting two columns instead of one requires scanning the table twice, doubling the estimated bytes processed.

In [ ]:
query = """
# Will use 155.12MiB
SELECT PULocationID
FROM `kestra-sandbox-485712.zoomcamp.yellow_tripdata_01_to_06_2024`;

# Will use 310.24MiB
SELECT PULocationID, DOLocationID
FROM `kestra-sandbox-485712.zoomcamp.yellow_tripdata_01_to_06_2024`;

"""

client_query.query(query).to_dataframe()

---

#### Question 4:
- How many records have a fare_amount of 0?

In [15]:
query = """

SELECT COUNT(*) AS total_rows
FROM `kestra-sandbox-485712.zoomcamp.yellow_tripdata_01_to_06_2024`
WHERE fare_amount = 0;

"""

client_query.query(query).to_dataframe()

,total_rows
0,8333


---

#### Question 5:
- What is the best strategy to make an optimized table in Big Query if your query will always filter based on tpep_dropoff_datetime and order the results by VendorID 
- (Create a new table with this strategy)



- Answer: Partition by tpep_dropoff_datetime and Cluster on VendorID

In [ ]:
query = """

CREATE OR REPLACE TABLE kestra-sandbox-485712.zoomcamp.yellow_tripdata_01_to_06_2024_partitioned_clustered
PARTITION BY DATE(tpep_dropoff_datetime)
CLUSTER BY VendorID AS
SELECT * FROM kestra-sandbox-485712.zoomcamp.yellow_tripdata_01_to_06_2024;

"""

client_query.query(query).to_dataframe()

---


#### Question 6:
- Write a query to retrieve the distinct VendorIDs between tpep_dropoff_datetime 2024-03-01 and 2024-03-15 (inclusive)
- Use the materialized table you created earlier in your from clause and note the estimated bytes. 
- Now change the table in the from clause to the partitioned table you created for question 5 and note the estimated bytes processed. 
- What are these values?

- Answer: 310.24 MB for non-partitioned table and 26.84 MB for the partitioned table

In [ ]:
query = """

# Original Table 310.24MiB
SELECT COUNT(DISTINCT VendorID)
FROM kestra-sandbox-485712.zoomcamp.yellow_tripdata_01_to_06_2024
WHERE DATE(tpep_dropoff_datetime) >= '2024-03-01' AND DATE(tpep_dropoff_datetime) <= '2024-03-15';

# Partioned and Clustered Table 26.84MiB
SELECT COUNT(DISTINCT VendorID)
FROM kestra-sandbox-485712.zoomcamp.yellow_tripdata_01_to_06_2024_partitioned_clustered
WHERE DATE(tpep_dropoff_datetime) >= '2024-03-01' AND DATE(tpep_dropoff_datetime) <= '2024-03-15';

"""

client_query.query(query).to_dataframe()

---

#### Question 7:
Where is the data stored in the External Table you created?
- Big Query
- Container Registry
- GCP Bucket
- Big Table

- Answer: GCP Bucket

---

#### Question 8:
It is best practice in Big Query to always cluster your data:
- True
- False

- Answer: False

---

#### (Bonus: Not worth points) Question 9:
- No Points: Write a SELECT count(*) query FROM the materialized table you created. 
- How many bytes does it estimate will be read? Why?

- Answer: This query will process 0 B when run because BigQuery can satisfy the query without scanning storage

In [ ]:
query = """

# Partioned and Clustered Table 26.84MiB
SELECT COUNT(*)
FROM kestra-sandbox-485712.zoomcamp.yellow_tripdata_01_to_06_2024_partitioned_clustered;

"""

client_query.query(query).to_dataframe()